In [1]:
from pathlib import Path

import pandas as pd

In [2]:
labels = {
	'stations': {
		'SE': 'sureste',
		'NE': 'noreste',
		'CE': 'centro',
		'NO': 'noroeste',
		'SO': 'suroeste',
		'NO2': ['noroeste2', 'noroeste 2'],
		'NTE': 'norte',
		'NE2': ['noreste2', 'noreste 2'],
		'SE2': ['sureste2', 'sureste 2'],
		'SO2': ['suroeste2', 'suroeste 2'],
		'SUR': 'sur',
		'NTE2': ['norte2', 'norte 2'],
		'SE3': ['sureste3', 'sureste 3'],
		'NE3': ['noreste3', 'noreste 3'],
		'NO3': ['noroeste3', 'noroeste 3']
	},
	'contaminants': {
		'PM10': 'Partículas menores a 10 micras',
		'PM2.5': 'Partículas menores a 2.5 micras',
		'O3': 'Ozono',
		'SO2': 'Dióxido de azufre',
		'NO2': 'Dióxido de nitrógeno',
		'CO': 'Monóxido de carbono',
		'NO': 'Monóxido de nitrógeno',
		'NOX': 'Óxidos de nitrógeno'
	}
}

additional_labels = {
	'parameters': {
		'TOUT': 'Temperatura',
		'RH': 'Humedad Relativa',
		'SR': 'Radiación Solar',
		'RAINF': 'Precipitación',
		'PRS': 'Presión Atmosférica',
		'WSR': 'Velocidad del Viento',
		'WDR': 'Dirección del Viento'
    }
}

In [3]:
# Dataset one
df_2020_2021_all_stations = pd.read_excel(
	Path("../data/raw/DATOS HISTÓRICOS 2020_2021_TODAS ESTACIONES.xlsx"),
	sheet_name=None
)

In [4]:
# Process dataset 1
frames = []
for name, frame in df_2020_2021_all_stations.items():
	if name == 'NOROESTE3':
		continue
	for code, codename in labels['stations'].items():
		frame_copy = frame.copy()
		if isinstance(codename, list):
			if any(name.upper() == cn.upper() for cn in codename):
				frame_copy['station_code'] = code
				frames.append(frame_copy)
		else:
			if name.upper() == codename.upper():
				frame_copy['station_code'] = code
				frames.append(frame_copy)


df_2020_2021_all_stations_processed = pd.concat(frames, ignore_index=True)

In [5]:
# Dataset two
df_2022_2023_all_stations = pd.read_excel(
	Path("../data/raw/DATOS HISTÓRICOS 2022_2023_TODAS ESTACIONES.xlsx"),
	sheet_name=None
)

In [6]:
# Process dataset 2
frames = []
for name, frame in df_2022_2023_all_stations.items():
	for code, codename in labels['stations'].items():
		frame_copy = frame.copy()
		if isinstance(codename, list):
			if any(name.upper() == cn.upper() for cn in codename):
				frame_copy['station_code'] = code
				frames.append(frame_copy)
		else:
			if name.upper() == codename.upper():
				frame_copy['station_code'] = code
				frames.append(frame_copy)


df_2022_2023_all_stations_processed = pd.concat(frames, ignore_index=True)

In [7]:
# Dataset three
df_2023_2024_all_stations = pd.read_excel(
	Path("../data/raw/DATOS HISTÓRICOS 2023_2024_TODAS ESTACIONES_ITESM.xlsx"),
	sheet_name='Param_horarios_Estaciones',
	header=None
)

In [8]:
# Process dataset 3
stations_map = labels['stations']

station_name_to_code = {}
for code, names in stations_map.items():
    if isinstance(names, list):
        for name in names:
            station_name_to_code[name.upper()] = code
    else:
        station_name_to_code[names.upper()] = code

stations_row = df_2023_2024_all_stations.iloc[0, 1:].astype(str).str.strip()
vars_row = df_2023_2024_all_stations.iloc[1, 1:].astype(str).str.strip()

full_body = df_2023_2024_all_stations.iloc[3:].reset_index(drop=True)
dates = pd.to_datetime(full_body.iloc[:, 0], errors="coerce", dayfirst=True)
body = full_body.iloc[:, 1:]


contaminants = list(labels["contaminants"].keys())

In [9]:
frames = []
for station in stations_row.unique():
    if pd.isna(station) or station == "nan" or station.upper() not in station_name_to_code:
        continue

    # Find which columns contain data for the current station
    station_columns = stations_row[stations_row == station].index.tolist()

    station_data = {
        "station_code": station_name_to_code[station.upper()],
        "date": dates
    }

    for col_idx in station_columns:
        if col_idx not in vars_row.index or col_idx not in body.columns:
            print(f"Warning: Column label {col_idx} not found in vars_row or body.")
            continue

        var_name = vars_row.loc[col_idx]
        # Normalize known aliases in dataset 3 (only)
        if var_name == "WDV":
            var_name = "WDR"

        if var_name in contaminants or var_name in additional_labels["parameters"]:
            column_data = body.loc[:, col_idx]
            station_data[var_name] = pd.to_numeric(column_data, errors='coerce')

    if len(station_data) > 3:
        station_df = pd.DataFrame(station_data)
        frames.append(station_df)


df_2023_2024_all_stations_processed = pd.concat(frames, ignore_index=True)

if 'date' in df_2023_2024_all_stations_processed.columns:
    mask_not_2024 = df_2023_2024_all_stations_processed['date'].isna() | (df_2023_2024_all_stations_processed['date'].dt.year != 2024)
    df_2023_2024_all_stations_processed_no_2024 = df_2023_2024_all_stations_processed.loc[mask_not_2024].reset_index(drop=True)
else:
    df_2023_2024_all_stations_processed_no_2024 = df_2023_2024_all_stations_processed

In [10]:
# Dataset four
df_2024_all_stations = pd.read_excel(
    Path("../data/raw/BD 2024.xlsx"),
    sheet_name=None
)

In [11]:
frames_2024 = []
for sheet_name, frame in df_2024_all_stations.items():
    code_clean = str(sheet_name).strip().upper()
    if code_clean in labels['stations'].keys():
        f = frame.copy()
        f['station_code'] = code_clean
        frames_2024.append(f)

df_2024_all_stations_processed = pd.concat(frames_2024, ignore_index=True) if frames_2024 else pd.DataFrame()

In [12]:
# Dataset five
df_2025_all_stations = pd.read_excel(
    Path("../data/raw/BD 2025.xlsx"),
    sheet_name=None
)

In [13]:
frames_2025 = []
for sheet_name, frame in df_2025_all_stations.items():
    code_clean = str(sheet_name).strip().upper()
    if code_clean in labels['stations'].keys():
        f = frame.copy()
        f['station_code'] = code_clean
        frames_2025.append(f)

df_2025_all_stations_processed = pd.concat(frames_2025, ignore_index=True) if frames_2025 else pd.DataFrame()

In [14]:
# Concat all dataframes
main_dataframe = pd.concat(
    [
        df_2020_2021_all_stations_processed,
        df_2022_2023_all_stations_processed,
        df_2023_2024_all_stations_processed_no_2024,
        df_2024_all_stations_processed,
        df_2025_all_stations_processed
    ],
    ignore_index=True
)

In [15]:
main_dataframe

,date,CO,NO,NO2,NOX,O3,PM10,PM2.5,PRS,RAINF,...,PM10 (ug/m3),PM2.5 (ug/m3),PRS (mmHg),RAINF (mm/h),RH (%),SO2 (ppb),SR (kW/m2),TOUT (ºC),WSR (km/h),WDR (azimutal)
0,2020-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,66.0,54.23,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-01-01 01:00:00,2.11,NaN,NaN,NaN,19.0,57.0,NaN,735.7,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-01-01 02:00:00,2.06,NaN,NaN,NaN,19.0,68.0,53.84,734.8,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-01-01 03:00:00,1.96,NaN,NaN,NaN,19.0,68.0,36.47,734.2,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-01-01 04:00:00,1.98,NaN,NaN,NaN,16.0,48.0,33.59,733.9,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
779391,2025-06-30 19:00:00,NaN,NaN,NaN,NaN,18.9,6.0,NaN,729,0.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
779392,2025-06-30 20:00:00,NaN,NaN,NaN,NaN,16.2,2.0,NaN,729,0.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
779393,2025-06-30 21:00:00,NaN,NaN,NaN,NaN,17.3,7.0,NaN,730,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
779394,2025-06-30 22:00:00,NaN,NaN,NaN,NaN,19.7,11.0,NaN,730,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
Path("../data/processed").mkdir(parents=True, exist_ok=True)

main_dataframe.to_csv(
	Path("../data/processed/main_dataframe.csv"),
	index=False
)